In [4]:
using ImpactChron

In [12]:
solarage = 4567.3
maxage = 2000
Δt = 1.
downscalefactor = 10
timelimit = 2569. # ceil(solarage-maxage,digits=-1)-1

2569.0

In [13]:
priordata = ImpactChron.readdlm("data/ArArages.csv",',')
ages_ = priordata[:,4]
iages = ages_ .> maxage
ages = (mu = float.(ages_[iages]), sig = float.(priordata[:,5])[iages])

(mu = [4529.257392313279, 4452.704274394391, 4298.36, 4533.0, 4467.5, 3704.0, 4246.4, 4407.0, 3630.0, 4508.0  …  2288.0, 4530.0, 4390.0, 4513.0, 4220.7, 4484.7, 4463.0, 4499.4, 4460.0, 4300.0], sig = [19.174306531075093, 28.720419853859617, 13.45, 6.0, 9.7, 17.7, 47.8, 71.0, 70.0, 9.0  …  69.0, 20.0, 219.5, 17.0, 8.8, 3.5, 10.0, 5.0, 11.2, 70.0])

In [14]:
types = PetroTypes((T3=600+273.,T4=700+273.,T5=800+273.,T6=950+273.), string.(priordata[iages,3]))

PetroTypes{Float64}(ImpactChron.TempProp{Float64}(873.0, 0.15053763440860216), ImpactChron.TempProp{Float64}(973.0, 0.21505376344086022), ImpactChron.TempProp{Float64}(1073.0, 0.27956989247311825), ImpactChron.TempProp{Float64}(1223.0, 0.3548387096774194), true)

In [20]:
impactsite=ImpactSite(Cone,C=0.01)

ImpactSite{Cone{Float64}, Float64}(Cone{Float64}(0.0, 0.0), 0.01)

In [15]:
ϕ = ( tss=solarage, rAlo=5.23e-5, Tm=log(250), R=log(150e3), ta=log(2.0), cAl=log(0.011), ρ=log(3210), Cp=log(900), k=log(3), Tc=log(500),
    tχα=0., τχα=10., Fχα=10., 
    tχβ=5., τχβ=10., Fχβ=10., 
    tχγ=0., τχγ=10., Fχγ=0.)

ϕσ = ( tss=.08, rAlo=0.065e-5, Tm=0.47,R=0.16, ta=.07, cAl=0.13, ρ=0.05, Cp=0.08, k=0.6, Tc=0.2, tχα=1., τχα=1., Fχα=1., tχβ=1., τχβ=1., Fχβ=1., tχγ=1., τχγ=1., Fχγ=1. )

paramdist = (
    tss = Nrm(4567.3,0.08), rAlo= Nrm(5.23e-5,0.065e-5), R = lNrm(11.920, 0.1869), ta = lNrm(0.6983, 0.0792), cAl = lNrm(-4.5665, 0.1316), Tm = lNrm(5.3517, 0.4691), Tc = lNrm(6.1914, 0.2571), ρ = lNrm(8.1199, 0.03996), Cp = lNrm(6.732, 0.07899), k = lNrm(0.3319, 0.6335),
    tχα  = Unf(0.,ϕ.tss-maxage), τχα  = Unf(0.,ϕ.tss), Fχα  = Unf(0.,1e4),
    tχβ  = Unf(0.,ϕ.tss-maxage), τχβ  = Unf(0.,ϕ.tss), Fχβ  = Unf(0.,1e4), 
    tχγ  = Unf(0.,ϕ.tss-maxage), τχγ  = Unf(0.,ϕ.tss), Fχγ  = Unf(0.,1e4))

vars = (:rAlo,:R,:ta,:cAl,:Tm,:Tc,:ρ,:Cp,:k,:τχα,:Fχα,:tχβ,:τχβ,:Fχβ) # Excluding :tχα so it stays fixed at tχα = 0. (4567.3 Ma) as a primordial flux.

(:rAlo, :R, :ta, :cAl, :Tm, :Tc, :ρ, :Cp, :k, :τχα, :Fχα, :tχβ, :τχβ, :Fχβ)

In [25]:
out = thermochron_metropolis(ϕ, ϕσ, vars, ages.mu, ages.sig, impactsite, plims=paramdist, petrotypes=types, burnin=0, nsteps=100, Δt= Δt, downscale=downscalefactor,Tmin=0., Tmax=1373., tmax=timelimit, nᵣ=200)

===  BURN IN COMPLETE  ===


Post-Burn-In Status:
---------------------------
run time: 0.0 minutes 

ll=-1895.861963109306 

rAlo → 5.23e-5
R → 11.918390573078392
ta → 0.6931471805599453
cAl → -4.509860006183766
Tm → 5.521460917862246
Tc → 6.214608098422191
ρ → 8.07402621612406
Cp → 6.802394763324311
k → 1.0986122886681098
τχα → 10.0
Fχα → 10.0
tχβ → 5.0
τχβ → 10.0
Fχβ → 10.0
---------------------------
== == == == == == == == ==
Now beginning 100 Markov chain iterations...


missing petrologic types: 

(meltdown) rejected


(meltdown) rejected


(meltdown) rejected


(meltdown) rejected


(meltdown) rejected


(meltdown) rejected


Dict{Symbol, Any} with 22 entries:
  :τχα    => [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0  …  10…
  :tss    => 4567.3
  :Fχα    => [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0  …  11…
  :cAl    => [-4.50986, -4.50986, -4.50986, -4.50986, -4.50986, -4.50986, -4.50…
  :Tc     => [6.21461, 6.21461, 6.21461, 6.21461, 6.21461, 6.21461, 6.21461, 6.…
  :tχα    => 0.0
  :Cp     => [6.80239, 6.80239, 6.80239, 6.80239, 6.80239, 6.80239, 6.80239, 6.…
  :R      => [11.9184, 11.9184, 11.9184, 11.9184, 11.9184, 11.9184, 11.9184, 11…
  :rAlo   => [5.23e-5, 5.23e-5, 5.23e-5, 5.23e-5, 5.23e-5, 5.23e-5, 5.23e-5, 5.…
  :Tm     => [5.52146, 5.52146, 5.52146, 5.52146, 5.52146, 5.52146, 5.52146, 5.…
  :k      => [1.09861, 1.09861, 1.09861, 1.09861, 1.09861, 1.09861, 1.09861, 1.…
  :τχβ    => [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0  …  10…
  :accept => Bool[0, 0, 1, 0, 0, 0, 0, 0, 0, 1  …  0, 0, 0, 0, 0, 1, 1, 0, 1, 1]
  :tχγ    => 0.0
  :prt    => [:k, :τ